In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import minimize, Bounds, LinearConstraint
from itertools import combinations


In [3]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP1-Gestion-de-pf/5_Industry_Portfolios.CSV', header=6)

df_5 = df.iloc[1108:1169]

df_5 = df_5.rename(columns={'Unnamed: 0': 'Date'})

df_5['Date'] = pd.to_datetime(df_5['Date'], format='%Y%m')

df_5.set_index('Date', inplace=True)

df_5 =df_5.apply(pd.to_numeric, errors='coerce')

df_10 = df.iloc[1049:1169]
ret = df_5.values / 100
variance = np.var(ret, axis=0, keepdims=True)
std = np.sqrt(np.var(ret, axis=0, keepdims=True))

# Plage des rendements cibles
target_returns = np.linspace(-0.10, 0.10, num=100)  # Génère 100 valeurs entre -10% et 10%

rf = 0.44 / 100

# Créer la liste portfolio_names à partir des noms de colonnes, en excluant la première colonne
portfolio_names = df.columns[1:].tolist()

# Définissons la matrice de covariance et le vecteur de rendements moyens
Sigma = np.cov(ret.T)

z_barre = np.mean(ret, axis=0)  
Z_barre = z_barre  # Rendements espérés des actifs
ones = np.ones((len(Z_barre))) # Vecteur de 1

# Rendement espéré du portefeuille cible
R_cible = 0.10  # Rendement espéré du portefeuille cible


In [4]:
N = 5  # Nombre total d'actifs
min_num_assets = 3
max_num_assets = 3
min_weight = 0.05  # poids minimum de 5%
target_return = 0.10
R_cible_basse = 0.01  # Rendement cible minimum de 5%
R_cible_haute = 0.10  # Rendement cible maximum de 15%

In [5]:
# Fonction objectif : Variance du portefeuille
def portfolio_variance(W,Sigma):
    return W.T @ (Sigma @ W)

# Poids initiaux : répartition uniforme
W = np.ones(len(Z_barre)) * (1.0 / len(Z_barre))

# Fonction d'optimisation
def optimize(func, W, Sigma, target_return):

    # Contraintes
    opt_constraints = ({'type': 'eq', 'fun': lambda W: ones @ W.T - 1},  # La somme des poids doit être égale à 1
                       {'type': 'eq', 'fun': lambda W: W.T @ z_barre - target_return})  # Le rendement attendu doit être égal au rendement cible

    # Optimisation
    optimal_weights = minimize(func, W, args=( Sigma), method='trust-constr', constraints=opt_constraints)

    return optimal_weights.x

# Appel de la fonction d'optimisation
optimal_weights = optimize(portfolio_variance, W, Sigma, R_cible)

print("Poids optimaux du portefeuille :", optimal_weights)

Poids optimaux du portefeuille : [-9.63824458  1.83033794 15.78561241 -3.96809407 -3.0096117 ]


/Users/dominicprenovost/anaconda3/envs/HECFinance/lib/python3.11/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [6]:
from scipy.optimize import minimize
import numpy as np
from itertools import combinations

# Résultats initiaux
best_variance = np.inf
best_weights = None
best_combination = None

# Générer toutes les combinaisons de 3 portefeuilles parmi 5
for combination in combinations(range(N), 3):
    Sigma_sub = Sigma[np.ix_(combination, combination)]
    Z_barre_sub = Z_barre[np.array(combination)]  # Assurez-vous que Z_barre_sub est correctement défini

    initial_weights = np.ones(len(combination)) / len(combination)

    # Définissez les contraintes en utilisant Z_barre_sub
    current_constraints = [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Les poids doivent sommer à 1
        {'type': 'ineq', 'fun': lambda weights: np.dot(weights, Z_barre_sub) - R_cible_basse},
        {'type': 'ineq', 'fun': lambda weights: R_cible_haute - np.dot(weights, Z_barre_sub)}  # Utilisez np.dot ici pour la compatibilité dimensionnelle
    ]

    # Lancer l'optimisation avec Sigma_sub et Z_barre_sub
    result = minimize(portfolio_variance, initial_weights, args=(Sigma_sub,), method='trust-constr', constraints=current_constraints)

    if result.success and result.fun < best_variance:
        best_variance = result.fun
        best_weights = result.x
        best_combination = [portfolio_names[i] for i in combination]  # Mappage des indices aux noms de portefeuilles

# Affichage des résultats
if best_combination is not None:
    print("Meilleure combinaison d'actifs:", best_combination)
    print("Poids optimaux:", best_weights)
    print("Variance minimale du portefeuille:", best_variance)

Meilleure combinaison d'actifs: ['HiTec', 'Hlth ', 'Other']
Poids optimaux: [ 0.36663048  0.75562886 -0.12225933]
Variance minimale du portefeuille: 0.0023281872906225874
